# DATA.STAT.840 Statistical Methods for Text Data Analysis
Exercises for Lecture 5: N-grams
Daniel Kusnetsoff

# Exercise 5.3: More adventures of Robin Hood, and a new journey to Mars.

In [ ]:
import requests
import bs4
import nltk
import numpy as np
#from nltk import word_tokenize, sent_tokenize
#import nltk.lm
nltk.download('nltk.lm')
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
import nltk.lm
#from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
#%% Get the text content of the page
def getpagetext(parsedpage):
    # Remove HTML elements that are scripts
    scriptelements=parsedpage.find_all('script')
    # Concatenate the text content from all table cells
    for scriptelement in scriptelements:
        # Extract this script element from the page.
        # This changes the page given to this function!
        scriptelement.extract()
    pagetext=parsedpage.get_text()
    return(pagetext)

In [ ]:
import scipy

def download_specific_ebook(ebook_url):
    ebook_page = requests.get(ebook_url)
    parsed_page = bs4.BeautifulSoup(ebook_page.content, 'html.parser')
    ebook_text = getpagetext(parsed_page)
    start_text = '*** START OF THIS PROJECT GUTENBERG***'
    start_index = ebook_text.find(start_text)
    end_index = ebook_text.find('*** END OF THE PROJECT GUTENBERG EBOOK')
    ebook_text = ebook_text[start_index + len(start_text):end_index]
    
    # remove whitespaces
    ebook_text = ebook_text.strip()
    ebook_text = ' '.join(ebook_text.split())
    return(ebook_text)

In [ ]:
robinHood_text = download_specific_ebook('https://www.gutenberg.org/files/10148/10148.txt')

In [ ]:
martianOdyssey_text = download_specific_ebook('https://www.gutenberg.org/files/23731/23731.txt')

In [ ]:
import nltk


In [ ]:
# tokenize text
robinHood_tokenized_text = nltk.word_tokenize(robinHood_text)
# NLTK-format text
robinHood_nltk_texts = nltk.Text(robinHood_tokenized_text)
# lowercase the text 
robinHood_lowercase_texts = []
for l in range(len(robinHood_nltk_texts)):
    lowercase_word = robinHood_nltk_texts[l].lower()
    robinHood_lowercase_texts.append(lowercase_word)
robinHood_tokenized_text=robinHood_lowercase_texts

In [ ]:
robinHood_tokenized_text[:20]

In [ ]:
# tokenize text
martianOdyssey_tokenized_text = nltk.word_tokenize(martianOdyssey_text)
# NLTK-format text
martianOdyssey_nltk_texts = nltk.Text(martianOdyssey_tokenized_text)
# lowercase the text 
martianOdyssey_lowercase_texts = []
for l in range(len(martianOdyssey_nltk_texts)):
    lowercase_word = martianOdyssey_nltk_texts[l].lower()
    martianOdyssey_lowercase_texts.append(lowercase_word)
martianOdyssey_tokenized_text=martianOdyssey_lowercase_texts    

In [ ]:
martianOdyssey_tokenized_text[:50]

In [ ]:
#%% Find the vocabulary, in a distributed fashion
robinHood_vocabularies=[]
robinHood_indices_in_vocabularies=[]
# Find the vocabulary of each document
for k in range(len(robinHood_tokenized_text)):
    # Get unique words and where they occur
    temptext=robinHood_tokenized_text[k]
    uniqueresults=np.unique(temptext,return_inverse=True)
    uniquewords=uniqueresults[0]
    wordindices=uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    robinHood_vocabularies.append(uniquewords)
    robinHood_indices_in_vocabularies.append(wordindices)
robinHood_vocabularies[0]

In [ ]:
robinHood_vocabularies[:10]

In [ ]:
#%% Find the vocabulary, in a distributed fashion
martianOdyssey_vocabularies=[]
martianOdyssey_indices_in_vocabularies=[]
# Find the vocabulary of each document
for k in range(len(martianOdyssey_tokenized_text)):
    # Get unique words and where they occur
    temptext=martianOdyssey_tokenized_text[k]
    uniqueresults=np.unique(temptext,return_inverse=True)
    uniquewords=uniqueresults[0]
    wordindices=uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    martianOdyssey_vocabularies.append(uniquewords)
    martianOdyssey_indices_in_vocabularies.append(wordindices)
martianOdyssey_vocabularies[0]

In [ ]:
martianOdyssey_vocabularies[:10]

# b)

In [ ]:
#import nltk.lm

In [ ]:
def n_gram_model(maxN, robinHood_tokenized_text):
    # Create N-gram training data
    ngramtraining_data, added_sentences = nltk.lm.preprocessing.padded_everygram_pipeline(maxN, robinHood_tokenized_text)
    # Create the maximum-likelihood n-gram estimate
    ngrammodel = nltk.lm.MLE(maxN)
    ngrammodel.fit(ngramtraining_data, padded_sentences)
    return(ngrammodel)
robinHood_tokenized_text[:10]

In [ ]:
#from nltk.tokenize.treebank import TreebankWordDetokenizer
#detokenize = TreebankWordDetokenizer().detokenize

def para_ngram(ngram_model, n_words):
    content = []
    for token in ngram_model.generate(n_words):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)
    

def build_ngram(textindexvector,n_vocab,maxN):
    # Create the overall structure that will store the n-gram
    allprobstructs_NextCount=[]
    allprobstructs_NextProb=[]
    allprobstructs_PreviousStruct=[]
    # Create unigram probability table, store it as the root of probtables
    tempstruct_NextCount=scipy.sparse.dok_matrix((n_vocab,1))
    tempstruct_NextProb=scipy.sparse.dok_matrix((n_vocab,1))
    tempstruct_PreviousStruct=scipy.sparse.dok_matrix((n_vocab,1))
    allprobstructs_NextCount.append(tempstruct_NextCount)
    allprobstructs_NextProb.append(tempstruct_NextProb)
    allprobstructs_PreviousStruct.append(tempstruct_PreviousStruct)
    nstructs=1
    # Count how many probability tables have been created at different
    # n-gram levels. Because this is a zero-based index, the index of the
    # level indicating how long a history each n-gram takes into account:
    # 0 for unigrams, 1 for bigrams, and so on.
    nstructsperlevel=np.zeros((maxN))
    # Initially there is only one table which is a unigram-table.
    nstructsperlevel[0]=1

# Iterate through the text
for t in range(len(textindexvector)):
    if (t%10)==0:
        print(str(t) + ' ' + str(nstructsperlevel))
    # Vocabulary index of the word at position t in the text
    tempword=textindexvector[t-0];
    # Suppose we have words w(1),...,w(t-3),w(t-2),w(t-1),w(t) in the text.
    # Then the transition to w(t) must be recorded for several n-grams:
    # []-->w(t) : unigram transition (n=1)
    # [w(t-1)]-->w(t) : bigram transition (n=2)
    # [w(t-2),w(t-1)]-->w(t) : trigram transition (n=3)
    # [w(t-3),w(t-2),w(t-1)]-->w(t) : 4-gram transition (n=4)
    # [w(t-4),w(t-3),w(t-2),w(t-1)]-->w(t) : 5-gram transition (n=5)
    # Start from the unigram (root of the tables), record the transition
    currentstruct=0
    # Record the transition into the transition counts:
    # and its count in the unigram model increases by 1.
    #allprobstructs[currentstruct]['Next'][tempword,0]=1
    allprobstructs_NextCount[currentstruct][tempword,0]+=1
    # Now record this transition into higher-level n-grams.
    # Address history up to maximum N-gram length or beginning of
    # the text, whichever is sooner.
    # Iterate a zero-based index "n" of steps back
    for n in range(min([maxN-1,t])):
        # Take the next step back.
        # Vocabulary index of the previous word
        previousword=textindexvector[t-n-1];
        # At this point in the for loop, the current probability table
        # allprobstructs[currentstruct] represents a (n+1)-gram which uses
        # a history of length (n): "[w(t-n),...,w(t-1)]--->NextWord".
        # The "previousword" w(t-n-1) is an expansion of it into a
        # (n+2)-gram which uses a history of length (n+1):
        # "[w(t-n-1),...,w(t-1)]--->NextWord". This expansion might exist
        # already or not. The field 'Previous' of the current (n+1)-gram
        # records whether that expansion exists.
        # Create a new history reference (next-level ngram) if it
        # did not exist.
        # Note that the unigram table has index 0, but it is never an
        # expansion of a smaller n-gram.
        if allprobstructs_PreviousStruct[currentstruct][previousword,0]==0:
            # Create the probability table for the expansion. Because this
            # is the first time this "[w(t-n-1),...,w(t-1)]--->NextWord" has
            # been needed, initially it has no next-words (the current
            # word will become its first observed next-word). Similarly,
            # because this is the first time this table is needed, it cannot
            # have any higher-level expansions yet.
            tempstruct_NextCount=scipy.sparse.dok_matrix((n_vocab,1));
            tempstruct_NextProb=scipy.sparse.dok_matrix((n_vocab,1));
            tempstruct_PreviousStruct=scipy.sparse.dok_matrix((n_vocab,1));
            # Add the created table into the overall list of all probability
            # tables, increase the count of tables overall and at the n-gram
            # level (history length n+1) where the table was created.
            nstructs+=1;
            nstructsperlevel[n+1]+=1;
            allprobstructs_NextCount.append(tempstruct_NextCount)
            allprobstructs_NextProb.append(tempstruct_NextProb)
            allprobstructs_PreviousStruct.append(tempstruct_PreviousStruct)
            # Mark that the expansion now exists into the current
            # current "[w(t-n),...,w(t-1)]--->NextWord" table
            # allprobstructs[currentstruct]['Previous'][previousword,0]=1
            # Add a pointer from the current table to the newly
            # created structure (index of the newly created table in the
            # overall list)
            allprobstructs_PreviousStruct \
            [currentstruct][previousword,0]=nstructs-1;
            # At this point we can be sure the next-level n-gram exists, so we
            # go to the next-level ngram and add the newest word-occurrence to it
            # as a possible next word, increasing its count.
        currentstruct=allprobstructs_PreviousStruct \
        [currentstruct][previousword,0];
        currentstruct=int(currentstruct)
        allprobstructs_NextCount[currentstruct][tempword,0]+=1
# For all tables that have been created, obtain their probabilities by
# normalizing their counts
for k in range(nstructs):
    allprobstructs_NextProb[k]=allprobstructs_NextCount[k] \
    /numpy.sum(allprobstructs_NextCount[k]);
return((allprobstructs_NextCount,allprobstructs_NextProb,allprobstructs_PreviousStruct))

In [ ]:
temptext=' '.join(robinHood_tokenized_text)
temptext=list(temptext)
# Create the "vocabulary" of the different individual characters
tempvocabulary=[]
myindices_in_tempvocabulary=[]
# Find the vocabulary of each document.
# Get unique words and where they occur
uniqueresults=np.unique(temptext,return_inverse=True)
tempvocabulary=uniqueresults[0]
tempindices=uniqueresults[1]

In [ ]:
temptext[:15]

import scipy.stats
def sample_ngram(allprobstructs,n_words_to_sample,maxN,initialtext):
    allprobstructs_NextProb=allprobstructs[1]
    allprobstructs_PreviousStruct=allprobstructs[2]
    sampletext=[]
    sampletext.extend(initialtext)
    for k in range(n_words_to_sample):
        # We are sampling a new word for position t
        t=len(initialtext)+k
        # Start from unigram probability table
        currentstruct=0
        # Try to use as much history as possible for sampling the next
        # word, but revert to smaller n-gram if data is not available for
        # the current history
        historycount=0
        for n in range(min([maxN-1,t])):
            # If we want, we can set a probability to use a higher-level n-gram
            usehigherlevel_probability=0.99
            if (scipy.stats.uniform.rvs() < usehigherlevel_probability):
                # Try to advance to the next-level n-gram
                historycount=historycount+1
                #print((t,historycount,len(sampletext)))
                previousword=sampletext[t-historycount]
                if allprobstructs_PreviousStruct[currentstruct][previousword,0]>0:
                    currentstruct=allprobstructs_PreviousStruct[currentstruct][previousword,0]
                    currentstruct=int(currentstruct)
                else:
                    # Don't try to advance any more times, just exist the for-loop
                    break
        # At this point we have found a probability table at some history level.
        # Sample from its nonzero entries.
        possiblewords=allprobstructs_NextProb[currentstruct].nonzero()[0]
        possibleprobs=numpy.squeeze(allprobstructs_NextProb[currentstruct][possiblewords,0].toarray(),axis=1)
        currentword=numpy.random.choice(possiblewords, p=possibleprobs)
        sampletext.append(currentword)
        # Return the created text
    return(sampletext)

In [ ]:
# Build the n-gram
maxN=1
myngram=n_gram_model(maxN, robinHood_tokenized_text)
# sample from the result
n_words_to_sample=800
initialtext=[]

In [ ]:
sampledtext=sample_ngram(myngram,n_words_to_sample,maxN,initialtext)
''.join(tempvocabulary[sampledtext])


In [ ]:
maxN=5
myngram=build_ngram(myindices_in_tempvocabulary[0],len(tempvocabulary[0]),maxN)
# This can be an array of vocabulary indices of previously observed words
initialtext=[]
# Sample a vector of word indices from the 5-gram
# following the initial text
n_words_to_sample=100
sampledtext=sample_ngram(myngram,n_words_to_sample,maxN,initialtext)
# Print the result
' '.join(myvocabularies[0][sampledtext])